# Анализ продаж сети магазинов

**Цель исследования** — проанализировать базу данных, в которой содержится информация об офлайн магазинах, товарных категориях и товарах, а также продажах торговой сети.\
\
**Задачи:**
1. Получить все возможные варианты магазин-товар.
2. Сделать выборку по продажам на определенную дату.
3. Рассчитать долю в суммарных продаж на день для направления Чистота.
4. Вывести информацию о топ-3 товарах по продажам в штуках в каждом магазине в каждую дату.
5. Найти сумму продаж в рублях за предыдущую дату только по магазинам СПб.
**Ход исследования:**
1. Ознакомиться с таблицами.
2. Решить поставленные задачи.
3. Вывести результаты каждого запроса.
4. Сделать выводы по каждой из задач.
**Описание данных**\
\
*Таблица shops*

Содержит данные о магазинах:\
`shopnumber` — номер магазина;\
`city` — город;\
`address` — адрес.

*Таблица goods*

Содержит данные о товарах:\
`id_good` — код товара;\
`gategory` — товарное направление;\
`good_name` — название товара;\
`price` — цена.

*Таблица sales*

Содержит данные о продажах:\
`date` — дата;\
`shopnumber` — номер магазина;\
`id_good` — код товара;\
`qty` — продано шт..


## Подключимся к базе данных, ознакомимся с таблицами

In [3]:
# импортируем библиотеки
import sqlite3
import pandas as pd 
from sqlalchemy import create_engine

In [4]:
# подключаемся к базе
engine = create_engine('sqlite:///db_shops_sales.db')

In [5]:
# выводим первые строки таблицы Goods

query = '''
           SELECT * FROM goods
           LIMIT 10
        '''
pd.io.sql.read_sql(query, con = engine)

,id_good,category,good_name,price
0,1234567,КРАСОТА,шамунь,100
1,1234568,ЧИСТОТА,стиральный порошок,120
2,1234569,ЧИСТОТА,чистящее средство,50
3,1234570,ЧИСТОТА,средство для мытья посуды,30
4,1234571,ДЕКОР,резинка для волос,20
5,1234572,ДЕКОР,тушь для ресниц,300
6,1234573,ДЕКОР,помада,200
7,1234574,ДЕКОР,подарочный пакет,100
8,1234575,КРАСОТА,мыло,10
9,1234576,ДЕКОР,лак для ногтей,120


In [7]:
# выводим первые строки таблицы Shops

query = '''
           SELECT * FROM shops
           LIMIT 10
        '''
pd.io.sql.read_sql(query, con = engine)

,shopnumber,city,address
0,1,СПб,"Ленина, 5"
1,2,МСК,"Пушкина, 10"
2,3,НВГ,"Ленина, 10"
3,4,МСК,"Лермонтова, 12"
4,5,НВГ,"Иванова, 56"
5,6,СПб,"Стачек, 125"
6,7,СПб,"Ветеранов, 105"
7,8,СПб,"Зеленина, 12"
8,9,МСК,"Центральная, 128"
9,10,НВГ,"Лермонтова, 18"


In [8]:
# выводим первые строки таблицы Sales

query = '''
           SELECT * FROM sales
           LIMIT 10
        '''
pd.io.sql.read_sql(query, con = engine)

,DATE,SHOPNUMBER,ID_GOOD,QTY
0,01.01.2016,1,1234569,100
1,01.01.2016,1,1234570,200
2,01.01.2016,1,1234571,300
3,01.01.2016,1,1234572,400
4,01.01.2016,1,1234573,500
5,01.01.2016,1,1234574,600
6,01.01.2016,1,1234575,700
7,01.01.2016,2,1234577,800
8,01.01.2016,2,1234578,900
9,01.01.2016,2,1234579,1000


## Найдем все возможные варианты магазин-товар

In [10]:
query = '''
           SELECT shopnumber AS 'Магазин', 
                  city AS 'Город', 
                  id_good AS 'Код товара', 
                  category AS 'Товарное направление'
           FROM shops, goods
        '''
pd.io.sql.read_sql(query, con = engine)

,Магазин,Город,Код товара,Товарное направление
0,1,СПб,1234567,КРАСОТА
1,1,СПб,1234568,ЧИСТОТА
2,1,СПб,1234569,ЧИСТОТА
3,1,СПб,1234570,ЧИСТОТА
4,1,СПб,1234571,ДЕКОР
...,...,...,...,...
235,16,МРМ,1234577,ДЕКОР
236,16,МРМ,1234578,КРАСОТА
237,16,МРМ,1234579,КРАСОТА
238,16,МРМ,1234580,КРАСОТА


In [ ]:
В нашей базе данных 16 магазинов и 15 товаров. Каждый магазин может составить пару с каждым товаром.
Получается, что 16∙15 = 240 вариантов магазин-товар.

## Сделаем выборку по продажам на 02.01.2016    

In [6]:
query = '''
           SELECT sales.shopnumber AS 'Магазин', 
                  city AS 'Город',
                  address AS 'Aдрес',
                  sum(qty) AS 'Сумма в шт',
                  sum(qty*price) AS 'Сумма в руб'
           FROM shops 
           LEFT JOIN sales ON shops.shopnumber=sales.shopnumber 
           LEFT JOIN goods ON sales.id_good=goods.id_good
           WHERE date='02.01.2016'
           GROUP BY sales.shopnumber
           ORDER BY sum(qty*price) DESC
        '''
pd.io.sql.read_sql(query, con = engine)

,Магазин,Город,Aдрес,Сумма в шт,Сумма в руб
0,6,СПб,"Стачек, 125",4000,552000
1,9,МСК,"Центральная, 128",5070,461825
2,11,СПб,"Молодежная, 108",4500,378500
3,14,МСК,"Советская, 2",3300,375000
4,3,НВГ,"Ленина, 10",3900,340500
5,15,МРМ,"Набережная, 5",2470,332350
6,10,НВГ,"Лермонтова, 18",2305,257050
7,2,МСК,"Пушкина, 10",1530,230200
8,4,МСК,"Лермонтова, 12",2380,224450
9,8,СПб,"Зеленина, 12",1255,189750


Наибольшая выручка на 02.01.2016 в магазине по адресу СПб, Стачек, 125. Наименьшая выручка - СПб, Ленина, 5.

## Рассчитаем долю в суммарных продажах на день для направления Чистота


In [36]:
query = ''' 
            WITH t AS (SELECT sales.date,
                              shops.city, 
                              goods.category,
                              CAST(SUM(goods.price*sales.qty) AS real) AS sales_clean
                       FROM shops 
                       LEFT JOIN sales ON shops.shopnumber=sales.SHOPNUMBER 
                       LEFT JOIN goods ON sales.id_good=goods.id_good
                       WHERE category="ЧИСТОТА"
                       GROUP BY sales.date, shops.city, goods.category), 
                 d AS (SELECT sales.date,
                              shops.city,
                              sum(goods.price*sales.qty) AS sales_all
                       FROM shops
                       LEFT JOIN sales ON shops.shopnumber=sales.SHOPNUMBER
                       LEFT JOIN goods ON sales.id_good=goods.id_good 
                       GROUP BY sales.date, shops.city)

            SELECT t.date AS 'Дата',
                   t.city AS 'Город', 
                   round(t.sales_clean/d.sales_all * 100, 1) AS 'Процент продаж'
            FROM t 
            JOIN d ON t.date=d.date AND t.city=d.city
            GROUP BY t.date, t.city
            ORDER BY 1, 3 DESC
        '''
pd.io.sql.read_sql(query, con = engine)

,Дата,Город,Процент продаж
0,01.01.2016,СПб,16.0
1,01.01.2016,МРМ,7.2
2,01.01.2016,НВГ,6.9
3,01.01.2016,МСК,2.6
4,02.01.2016,НВГ,28.1
5,02.01.2016,СПб,17.9
6,02.01.2016,МСК,12.9
7,02.01.2016,МРМ,10.1
8,03.01.2016,НВГ,41.2
9,03.01.2016,СПб,7.3


Наибольшая доля в суммарных продажах для направления Чистота:\
1 января - в Санкт-Петербурге, 16 %;\
2 января - в Нижнем Новгороде, 28,1%;\
3 января - в Нижнем Новгороде, 41,2 %.

## Выведем информацию о топ-3 товарах по продажам в штуках в каждом магазине в каждую дату


In [19]:
query = ''' WITH cte AS (SELECT s.date, 
                                s.shopnumber,
                                g.good_name,
                                s.qty,
                                DENSE_RANK() OVER (PARTITION BY s.shopnumber, s.date ORDER BY s.qty DESC) AS rank 
                         FROM sales AS s
                         LEFT JOIN goods AS g ON s.id_good=g.id_good
                         ORDER BY date)

            SELECT date AS 'Дата',
                   shopnumber AS 'Магазин',
                   good_name as 'Товар',
                   qty as 'Продано шт'
            FROM cte
            WHERE rank<=3
        '''

pd.io.sql.read_sql(query, con = engine)

,Дата,Магазин,Товар,Продано шт
0,01.01.2016,1,мыло,700
1,01.01.2016,1,подарочный пакет,600
2,01.01.2016,1,помада,500
3,01.01.2016,2,гель для душа,1000
4,01.01.2016,2,крем для рук,900
...,...,...,...,...
138,03.01.2016,15,средство для мытья посуды,195
139,03.01.2016,15,чистящее средство,180
140,03.01.2016,16,крем для лица,600
141,03.01.2016,16,гель для душа,500


Топ-3 товара по продажам в штуках в мазагине №1 - мыло (продано 700 шт), подарочные пакеты (продано 600 шт) и помада (продано 500 шт). 

## Найдем сумму продаж в рублях за предыдущую дату только по магазинам СПб


In [20]:
query = '''WITH t AS (SELECT date,
                             sales.shopnumber,
                             category,
                             sum(qty*price) AS sum_category
                      FROM shops 
                      JOIN sales ON shops.shopnumber=sales.shopnumber 
                      JOIN goods ON sales.id_good=goods.id_good
                      WHERE city="СПб"
                      GROUP BY date, sales.shopnumber, category)

           SELECT date AS 'Дата',
                  shopnumber AS 'Магазин', 
                  category AS 'Товарное направление',
                  sum_category AS 'Сумма в руб', 
                  IFNULL(LAG(sum_category, 1) OVER (PARTITION BY shopnumber, category ORDER BY date), 0) AS 'Сумма за пред. дату'
           FROM t
           ORDER BY date
        '''

pd.io.sql.read_sql(query, con = engine)

,Дата,Магазин,Товарное направление,Сумма в руб,Сумма за пред. дату
0,01.01.2016,1,ДЕКОР,286000,0
1,01.01.2016,1,КРАСОТА,7000,0
2,01.01.2016,1,ЧИСТОТА,11000,0
3,01.01.2016,6,ДЕКОР,186000,0
4,01.01.2016,6,КРАСОТА,26250,0
5,01.01.2016,6,ЧИСТОТА,41600,0
6,01.01.2016,7,ДЕКОР,436000,0
7,01.01.2016,7,КРАСОТА,46600,0
8,01.01.2016,8,ДЕКОР,4200,0
9,01.01.2016,8,КРАСОТА,35250,0


Сумма продаж в рублях направления ДЕКОР в магазине №6 составила 534 000 руб (02.01.2016 ),
а в предыдущую дату (01.01.2026) - 186 000 руб.

# Общие выводы 

Исследуя базу данных, нам удалось сделать следующие выводы:
 * В нашей базе данных 240 вариантов магазин-товар.
 * Наибольшая выручка на 02.01.2016 в магазине по адресу СПб, Стачек, 125. Наименьшая выручка - СПб, Ленина, 5.
 * Наибольшая доля в суммарных продажах для направления Чистота:\
    1 января - в Санкт-Петербурге, 16 %;\
    2 января - в Нижнем Новгороде, 28,1%;\
    3 января - в Нижнем Новгороде, 41,2 %.
 * Топ-3 товара по продажам в штуках в мазагине №1 - мыло (продано 700 шт), подарочные пакеты (продано 600 шт) и помада (продано 500 шт).
 * Сумма продаж в рублях направления ДЕКОР в магазине №6 составила 534 000 руб (02.01.2016 ), а в предыдущую дату (01.01.2026) - 186 000 руб.